In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정 (matplotlib 경고 방지)
plt.rcParams['axes.unicode_minus'] = False

print("=" * 60)
print("가상 회사 직원 데이터 분석 프로젝트")
print("=" * 60)

가상 회사 직원 데이터 분석 프로젝트


In [3]:
# 1. 데이터 생성
np.random.seed(42)
n_employees = 500

departments = ['개발', '마케팅', '영업', '인사', '재무']
positions = ['사원', '대리', '과장', '차장', '부장']

# 기본 정보
employee_data = {
    '사번': range(1001, 1001 + n_employees),
    '부서': np.random.choice(departments, n_employees, p=[0.3, 0.2, 0.25, 0.15, 0.1]),
    '직급': np.random.choice(positions, n_employees, p=[0.35, 0.25, 0.2, 0.12, 0.08]),
    '성별': np.random.choice(['남', '여'], n_employees),
    '나이': np.random.randint(25, 60, n_employees),
    '근속연수': np.random.randint(1, 30, n_employees)
}

df = pd.DataFrame(employee_data)

# 연봉 생성 (직급과 근속연수에 영향 받음)
position_salary_base = {'사원': 35000, '대리': 45000, '과장': 60000,
                        '차장': 75000, '부장': 95000}
df['연봉'] = df['직급'].map(position_salary_base) + df['근속연수'] * 1000 + np.random.normal(0, 5000, n_employees)
df['연봉'] = df['연봉'].clip(30000, 150000)  # 현실적인 범위로 제한

# 성과평가 (연봉과 약간의 상관관계)
df['성과평가'] = np.random.normal(3.5, 0.8, n_employees)
df['성과평가'] = df['성과평가'].clip(1, 5)

# 만족도 (연봉, 성과평가와 상관관계)
df['만족도'] = 50 + (df['연봉'] - df['연봉'].mean()) / 1000 + df['성과평가'] * 5 + np.random.normal(0, 10, n_employees)
df['만족도'] = df['만족도'].clip(0, 100)

# 연차 사용일수
df['연차사용'] = np.random.poisson(12, n_employees)

# 일부 결측치 추가
df.loc[np.random.choice(df.index, 20), '성과평가'] = np.nan
df.loc[np.random.choice(df.index, 10), '만족도'] = np.nan

print("\n1. 데이터 생성 완료!")
print(f"총 직원 수: {len(df)}명")
print(df.head(10))


1. 데이터 생성 완료!
총 직원 수: 500명
     사번   부서  직급 성별  나이  근속연수            연봉      성과평가         만족도  연차사용
0  1001  마케팅  과장  남  39     9  66942.895137  3.647921   61.318486    12
1  1002   재무  대리  여  49     6  53643.744229  2.657628   57.121860     9
2  1003   영업  사원  여  54    25  56543.413145  3.666119   63.885016    10
3  1004   영업  차장  여  38    13  76881.939845  4.347952   62.746379     8
4  1005   개발  과장  여  31    21  67429.463498  1.996351   57.531849    12
5  1006   개발  사원  남  41     2  33002.508334  2.548742   30.021594     6
6  1007   개발  차장  여  44     8  83781.890615  3.647851   87.061748    22
7  1008   인사  차장  남  33    21  97659.858670  3.868336  100.000000    10
8  1009   영업  부장  남  46     2  94566.214957       NaN  100.000000    11
9  1010   영업  과장  남  26     4  64709.175836  3.588712   66.286673    10


In [ ]:
# 2. 데이터 탐색
print("\n" + "=" * 60)
print("2. 데이터 구조 파악")
print("=" * 60)
print(df.info())

print("\n기초 통계:")
print(df.describe())

In [ ]:
# 3. 결측치 처리
print("\n" + "=" * 60)
print("3. 결측치 처리")
print("=" * 60)
print("결측치 현황:")
print(df.isnull().sum())

# 성과평가: 중앙값으로 대체
df['성과평가'].fillna(df['성과평가'].median(), inplace=True)
# 만족도: 평균으로 대체
df['만족도'].fillna(df['만족도'].mean(), inplace=True)

print("\n결측치 처리 완료!")

In [ ]:
# 4. 부서별 분석
print("\n" + "=" * 60)
print("4. 부서별 분석")
print("=" * 60)

dept_analysis = df.groupby('부서').agg({
    '사번': 'count',
    '연봉': ['mean', 'median', 'std'],
    '성과평가': 'mean',
    '만족도': 'mean',
    '나이': 'mean',
    '근속연수': 'mean'
}).round(2)

dept_analysis.columns = ['인원수', '평균연봉', '중앙연봉', '연봉표준편차',
                         '평균성과', '평균만족도', '평균나이', '평균근속']
print(dept_analysis)

In [ ]:
# 5. 직급별 분석
print("\n" + "=" * 60)
print("5. 직급별 분석")
print("=" * 60)

position_analysis = df.groupby('직급').agg({
    '사번': 'count',
    '연봉': ['mean', 'median'],
    '성과평가': 'mean',
    '만족도': 'mean'
}).round(2)

position_analysis.columns = ['인원수', '평균연봉', '중앙연봉', '평균성과', '평균만족도']
print(position_analysis)

In [ ]:
# 6. 상관관계 분석
print("\n" + "=" * 60)
print("6. 상관관계 분석")
print("=" * 60)

corr_cols = ['나이', '근속연수', '연봉', '성과평가', '만족도', '연차사용']
correlation_matrix = df[corr_cols].corr()
print(correlation_matrix)

In [ ]:
# 7. 이상치 탐지
print("\n" + "=" * 60)
print("7. 이상치 탐지 (연봉 기준)")
print("=" * 60)

Q1 = df['연봉'].quantile(0.25)
Q3 = df['연봉'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['연봉'] < lower_bound) | (df['연봉'] > upper_bound)]
print(f"이상치 개수: {len(outliers)}명")
print(f"이상치 비율: {len(outliers)/len(df)*100:.1f}%")
print("\n이상치 데이터:")
print(outliers[['사번', '부서', '직급', '연봉', '근속연수']].head(10))

In [1]:
# 8. 가설 검증: 부서별 만족도 차이가 있을까?
print("\n" + "=" * 60)
print("8. 통계적 분석: 부서별 만족도")
print("=" * 60)

dept_satisfaction = df.groupby('부서')['만족도'].agg(['mean', 'std', 'count'])
print(dept_satisfaction)


8. 통계적 분석: 부서별 만족도


NameError: name 'df' is not defined

In [ ]:
# 9. 종합 시각화
fig = plt.figure(figsize=(20, 12))

# 2x3 그리드로 6개의 subplot
# (1) 부서별 인원 분포
ax1 = plt.subplot(2, 3, 1)
df['부서'].value_counts().plot(kind='bar', ax=ax1, color='skyblue', edgecolor='black')
ax1.set_title('부서별 인원 분포', fontsize=14, fontweight='bold')
ax1.set_xlabel('부서')
ax1.set_ylabel('인원 수')
ax1.tick_params(axis='x', rotation=45)

# (2) 직급별 평균 연봉
ax2 = plt.subplot(2, 3, 2)
position_order = ['사원', '대리', '과장', '차장', '부장']
position_salary = df.groupby('직급')['연봉'].mean().reindex(position_order)
position_salary.plot(kind='bar', ax=ax2, color='lightcoral', edgecolor='black')
ax2.set_title('직급별 평균 연봉', fontsize=14, fontweight='bold')
ax2.set_xlabel('직급')
ax2.set_ylabel('연봉 ($)')
ax2.tick_params(axis='x', rotation=45)

# (3) 연봉 분포 히스토그램
ax3 = plt.subplot(2, 3, 3)
ax3.hist(df['연봉'], bins=30, edgecolor='black', alpha=0.7, color='lightgreen')
ax3.axvline(df['연봉'].mean(), color='r', linestyle='--', linewidth=2,
            label=f'평균: ${df["연봉"].mean():,.0f}')
ax3.axvline(df['연봉'].median(), color='g', linestyle='--', linewidth=2,
            label=f'중앙값: ${df["연봉"].median():,.0f}')
ax3.set_title('연봉 분포', fontsize=14, fontweight='bold')
ax3.set_xlabel('연봉 ($)')
ax3.set_ylabel('빈도')
ax3.legend()

# (4) 상관관계 히트맵
ax4 = plt.subplot(2, 3, 4)
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=1, ax=ax4, cbar_kws={"shrink": 0.8})
ax4.set_title('변수 간 상관관계', fontsize=14, fontweight='bold')

# (5) 근속연수 vs 연봉 산점도
ax5 = plt.subplot(2, 3, 5)
scatter = ax5.scatter(df['근속연수'], df['연봉'], c=df['만족도'],
                     cmap='viridis', alpha=0.6, s=50)
ax5.set_title('근속연수 vs 연봉 (색상: 만족도)', fontsize=14, fontweight='bold')
ax5.set_xlabel('근속연수 (년)')
ax5.set_ylabel('연봉 ($)')
plt.colorbar(scatter, ax=ax5, label='만족도')

# (6) 부서별 만족도 박스플롯
ax6 = plt.subplot(2, 3, 6)
df.boxplot(column='만족도', by='부서', ax=ax6)
ax6.set_title('부서별 만족도 분포', fontsize=14, fontweight='bold')
ax6.set_xlabel('부서')
ax6.set_ylabel('만족도')
plt.suptitle('')  # 기본 제목 제거

plt.tight_layout()
plt.show()

In [ ]:
# 10. 심화 분석
print("\n" + "=" * 60)
print("9. 심화 분석")
print("=" * 60)

# 성별 임금 격차 분석
print("\n【 성별 분석 】")
gender_analysis = df.groupby('성별').agg({
    '연봉': ['mean', 'median'],
    '성과평가': 'mean',
    '만족도': 'mean',
    '사번': 'count'
}).round(2)
gender_analysis.columns = ['평균연봉', '중앙연봉', '평균성과', '평균만족도', '인원수']
print(gender_analysis)

# 고성과자 vs 저성과자 비교
print("\n【 성과평가 구간별 분석 】")
df['성과구간'] = pd.cut(df['성과평가'], bins=[0, 2, 3, 4, 5],
                        labels=['하위', '중하위', '중상위', '상위'])
performance_analysis = df.groupby('성과구간').agg({
    '연봉': 'mean',
    '만족도': 'mean',
    '연차사용': 'mean',
    '사번': 'count'
}).round(2)
performance_analysis.columns = ['평균연봉', '평균만족도', '평균연차사용', '인원수']
print(performance_analysis)

# 연령대별 분석
print("\n【 연령대별 분석 】")
df['연령대'] = pd.cut(df['나이'], bins=[20, 30, 40, 50, 60],
                     labels=['20대', '30대', '40대', '50대'])
age_analysis = df.groupby('연령대').agg({
    '연봉': 'mean',
    '만족도': 'mean',
    '성과평가': 'mean',
    '사번': 'count'
}).round(2)
age_analysis.columns = ['평균연봉', '평균만족도', '평균성과', '인원수']
print(age_analysis)

In [ ]:
# 11. 주요 인사이트 도출
print("\n" + "=" * 60)
print("10. 주요 인사이트 및 권장사항")
print("=" * 60)

# 만족도가 낮은 부서 찾기
low_satisfaction_dept = dept_analysis['평균만족도'].idxmin()
high_satisfaction_dept = dept_analysis['평균만족도'].idxmax()

# 연봉 대비 만족도가 낮은 부서
df['연봉대비만족도'] = df['만족도'] / (df['연봉'] / 1000)
dept_efficiency = df.groupby('부서')['연봉대비만족도'].mean().sort_values()

# 이직 위험군 (만족도 하위 25%)
at_risk = df[df['만족도'] < df['만족도'].quantile(0.25)]

insights = f"""
【 핵심 발견사항 】

1. 인력 구성
   - 전체 직원: {len(df)}명
   - 가장 큰 부서: {df['부서'].value_counts().idxmax()} ({df['부서'].value_counts().max()}명)
   - 평균 연령: {df['나이'].mean():.1f}세
   - 평균 근속연수: {df['근속연수'].mean():.1f}년

2. 연봉 현황
   - 전체 평균 연봉: ${df['연봉'].mean():,.0f}
   - 연봉 중앙값: ${df['연봉'].median():,.0f}
   - 연봉 표준편차: ${df['연봉'].std():,.0f}
   - 최고 연봉: ${df['연봉'].max():,.0f}
   - 최저 연봉: ${df['연봉'].min():,.0f}

3. 만족도 분석
   - 전체 평균 만족도: {df['만족도'].mean():.1f}점
   - 만족도 최고 부서: {high_satisfaction_dept} ({dept_analysis.loc[high_satisfaction_dept, '평균만족도']:.1f}점)
   - 만족도 최저 부서: {low_satisfaction_dept} ({dept_analysis.loc[low_satisfaction_dept, '평균만족도']:.1f}점)
   - 이직 위험군 (하위 25%): {len(at_risk)}명

4. 상관관계
   - 연봉-만족도 상관계수: {correlation_matrix.loc['연봉', '만족도']:.3f}
   - 성과평가-만족도 상관계수: {correlation_matrix.loc['성과평가', '만족도']:.3f}
   - 근속연수-연봉 상관계수: {correlation_matrix.loc['근속연수', '연봉']:.3f}

【 권장사항 】

1. 만족도 개선 우선순위
   → {low_satisfaction_dept}부서의 만족도가 가장 낮음 ({dept_analysis.loc[low_satisfaction_dept, '평균만족도']:.1f}점)
   → 해당 부서 직원 면담 및 개선 방안 마련 필요

2. 이직 위험 관리
   → {len(at_risk)}명의 직원이 만족도 하위 25% (이직 위험군)
   → 이들의 평균 만족도: {at_risk['만족도'].mean():.1f}점
   → 조기 개입 및 1:1 면담 권장

3. 연봉 체계 검토
   → 연봉 표준편차가 ${df['연봉'].std():,.0f}로 편차가 큼
   → 직급별 연봉 가이드라인 재검토 필요
   → 성과평가와 연봉의 연계성 강화 고려

4. 부서별 맞춤 관리
   → 연봉 대비 만족도가 가장 낮은 부서: {dept_efficiency.index[0]}
   → 비금전적 보상 및 복리후생 개선 검토

5. 고성과자 유지
   → 상위 성과자의 평균 만족도: {df[df['성과구간']=='상위']['만족도'].mean():.1f}점
   → 핵심 인재 유지 프로그램 강화 필요
"""

print(insights)

In [ ]:
# 12. 추가 시각화
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# (1) 성과평가 구간별 연봉 분포
ax1 = axes[0, 0]
df.boxplot(column='연봉', by='성과구간', ax=ax1)
ax1.set_title('성과평가 구간별 연봉 분포', fontsize=12, fontweight='bold')
ax1.set_xlabel('성과 구간')
ax1.set_ylabel('연봉 ($)')
plt.suptitle('')

# (2) 연령대별 만족도
ax2 = axes[0, 1]
age_satisfaction = df.groupby('연령대')['만족도'].mean()
age_satisfaction.plot(kind='bar', ax=ax2, color='coral', edgecolor='black')
ax2.set_title('연령대별 평균 만족도', fontsize=12, fontweight='bold')
ax2.set_xlabel('연령대')
ax2.set_ylabel('만족도')
ax2.tick_params(axis='x', rotation=45)

# (3) 부서별 성과평가 분포
ax3 = axes[1, 0]
df.boxplot(column='성과평가', by='부서', ax=ax3)
ax3.set_title('부서별 성과평가 분포', fontsize=12, fontweight='bold')
ax3.set_xlabel('부서')
ax3.set_ylabel('성과평가')
plt.suptitle('')

# (4) 연차 사용일수 분포
ax4 = axes[1, 1]
ax4.hist(df['연차사용'], bins=range(0, 25), edgecolor='black', alpha=0.7, color='lightblue')
ax4.axvline(df['연차사용'].mean(), color='r', linestyle='--', linewidth=2,
            label=f'평균: {df["연차사용"].mean():.1f}일')
ax4.set_title('연차 사용일수 분포', fontsize=12, fontweight='bold')
ax4.set_xlabel('연차 사용일수')
ax4.set_ylabel('빈도')
ax4.legend()

plt.tight_layout()
plt.show()

In [ ]:
# 13. 최종 리포트 저장 (선택사항)
print("\n" + "=" * 60)
print("11. 분석 완료!")
print("=" * 60)
print("\n이번 프로젝트에서 사용한 기술:")
print("✓ NumPy: 배열 연산, 통계 계산")
print("✓ Pandas: 데이터프레임 조작, 그룹 분석")
print("✓ 기술통계: 평균, 중앙값, 표준편차, 사분위수")
print("✓ 상관관계 분석")
print("✓ 이상치 탐지 (IQR 방법)")
print("✓ 데이터 시각화: 히스토그램, 박스플롯, 산점도, 히트맵")
print("✓ 결측치 처리")
print("✓ 범주형 데이터 분석")

# 데이터를 CSV로 저장하고 싶다면 (선택사항):
# df.to_csv('employee_analysis.csv', index=False, encoding='utf-8-sig')
# print("\n데이터가 'employee_analysis.csv'로 저장되었습니다.")